In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    if b > a:
        b = a^b
        a = a^b
        b = a^b
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    if (a+b) > 100000:
        ans = str(a + b)
        q = '{}+{}'.format(a, b)
    else:
        ans = str(a - b)
        q = '{}-{}'.format(a, b)
        
    query = q + ' ' * (MAXLEN - len(q))
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['94-11  ', '57-1   ', '834-143', '64-60  ', '40-1   '] ['83  ', '56  ', '691 ', '4   ', '39  ']


In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 13)
(18000, 4, 13)
Validation Data:
(2000, 7, 13)
(2000, 4, 13)
Testing Data:
(60000, 7, 13)
(60000, 4, 13)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False  True False False False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [False False False  True False False False False False False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [ True False False False False False False False False False False
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False False False False False False  True
   False False]
  [False False False False False False False False False False  True
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [ True False False False False False False False Fals

In [12]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for iteration in range(10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 4s 246us/step - loss: 1.8941 - acc: 0.3276 - val_loss: 1.7369 - val_acc: 0.3546
Epoch 2/10
18000/18000 [==============================] - 3s 177us/step - loss: 1.7190 - acc: 0.3539 - val_loss: 1.7003 - val_acc: 0.3639
Epoch 3/10
18000/18000 [==============================] - 3s 183us/step - loss: 1.6601 - acc: 0.3817 - val_loss: 1.6224 - val_acc: 0.3945
Epoch 4/10
18000/18000 [==============================] - 3s 185us/step - loss: 1.5790 - acc: 0.4116 - val_loss: 1.5516 - val_acc: 0.4266
Epoch 5/10
18000/18000 [==============================] - 3s 191us/step - loss: 1.5183 - acc: 0.4392 - val_loss: 1.4941 - val_acc: 0.4385
Epoch 6/10
18000/18000 [==============================] - 3s 186us/step - loss: 1.4710 - acc: 0.4580 - val_loss: 1.4498 - val_acc: 0.4689
Epoch 7/10


Epoch 5/10
18000/18000 [==============================] - 5s 261us/step - loss: 0.5014 - acc: 0.8208 - val_loss: 0.5414 - val_acc: 0.7913
Epoch 6/10
18000/18000 [==============================] - 4s 202us/step - loss: 0.4665 - acc: 0.8353 - val_loss: 0.4958 - val_acc: 0.8132
Epoch 7/10
18000/18000 [==============================] - 4s 199us/step - loss: 0.4405 - acc: 0.8475 - val_loss: 0.5009 - val_acc: 0.8089
Epoch 8/10
18000/18000 [==============================] - 4s 218us/step - loss: 0.4110 - acc: 0.8606 - val_loss: 0.4366 - val_acc: 0.8440
Epoch 9/10
18000/18000 [==============================] - 7s 410us/step - loss: 0.3858 - acc: 0.8722 - val_loss: 0.4205 - val_acc: 0.8458
Epoch 10/10
18000/18000 [==============================] - 5s 303us/step - loss: 0.3607 - acc: 0.8831 - val_loss: 0.3837 - val_acc: 0.8635
Q 756-79  T 677  ☑ 677 
Q 218-43  T 175  ☑ 175 
Q 978-869 T 109  ☒ 100 
Q 851-25  T 826  ☑ 826 
Q 342-28  T 314  ☑ 314 
Q 55-8    T 47   ☑ 47  
Q 190-112 T 78   ☒ 90  
Q 6

18000/18000 [==============================] - 3s 189us/step - loss: 0.0502 - acc: 0.9887 - val_loss: 0.1093 - val_acc: 0.9659
Epoch 2/10
18000/18000 [==============================] - 5s 251us/step - loss: 0.0664 - acc: 0.9819 - val_loss: 0.1239 - val_acc: 0.9571
Epoch 3/10
14592/18000 [=======================>......] - ETA: 0s - loss: 0.0704 - acc: 0.9795

In [ ]:
Right = 0
Wrong = 0
for i in range(10000):
    ind = np.random.randint(0, len(test_x))
    rowx, rowy = test_x[np.array([i])], test_y[np.array([i])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    
    #print('Q', q[::-1] if REVERSE else q, end=' ')
    #print('T', correct, end=' ')

    if correct == guess:
        Right += 1
    else:
        Wrong += 1
print(Right/10000)